In [1]:
%load_ext autoreload
%load_ext autotime
%autoreload 2

# Online Outcomes: Predicting Success in Virtual Learning

**<font size="4">Author: Jeffrey Hanif Watson</font>**

## Imports and Project Path

In [55]:
# setting project path
import os, sys

gparent = os.path.join(os.pardir, os.pardir)
sys.path.append(gparent)

# importing packages
import os, sys
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

from src import class_Database_report as d
from src import helper_functions as f
from src import visualizations as v

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_theme('talk')
plt.style.use('fivethirtyeight')
sns.set_palette('GnBu_r')
pd.options.display.max_rows = 300

time: 3.15 ms


## Creating The Database

In [3]:
# creating database & database class instance
database_name = 'outcomes.db'
db = dr.Database(database_name)

time: 1.19 ms


**Run the cell below once to populate the database. After the database is populated, the cell can be commented out**

In [4]:
# file_name = 'anonymisedData.zip'
# db.populate(file_name, database_name)

time: 532 µs


## Exploring the Data